# MODEL-BASED META-LEARNING USING 

# Conditional Neural Processes

In [17]:
!pip install import_ipynb --quiet

In [18]:
!pip install learn2learn --quiet

In [19]:
import requests

In [1]:
r=requests.get("https://github.com/gmshroff/metaLearning2022/blob/main/code/utils.ipynb?raw=true")
with open('./utils.ipynb','wb') as f: f.write(r.content)

NameError: ignored

In [ ]:
r=requests.get("https://github.com/gmshroff/metaLearning2022/blob/main/code/l2lutils.ipynb?raw=true")
with open('./l2lutils.ipynb','wb') as f: f.write(r.content)

In [1]:
import import_ipynb
import utils
import l2lutils

importing Jupyter notebook from utils.ipynb
importing Jupyter notebook from l2lutils.ipynb


In [2]:
from IPython import display
import torch
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
from l2lutils import KShotLoader
from IPython import display
import torch.nn as nn
from time import time

This (CNP is the class you need to write for HW2

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%ls drive/MyDrive/'Colab Notebooks'/'Meta_Learning_hw2'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
CNP.ipynb          l2lutils.ipynb  models.ipynb  nb3-CNP.ipynb  utils.ipynb
course_data.ipynb  MAN.ipynb       nb2.ipynb     nb3-MAN.ipynb


In [3]:
!cp drive/MyDrive/'Colab Notebooks'/'Meta_Learning_hw2'/models.ipynb .
!cp drive/MyDrive/'Colab Notebooks'/'Meta_Learning_hw2'/CNP.ipynb .

In [4]:

from CNP import CNP
import models

# Data Generation/Loading

In [ ]:
#Generate data - euclidean
meta_train_ds, meta_test_ds, full_loader = utils.euclideanDataset(n_samples=10000,n_features=20,n_classes=10,batch_size=32)

In [ ]:
# Define an MLP network. Note that input dimension has to be data dimension. For classification
# final dimension has to be number of classes; for regression one.
#torch.manual_seed(10)
net = models.MLP(dims=[20,64,10])

In [ ]:
# Train the network; note that network is trained in place so repeated calls further train it.
net,losses,accs=models.Train(net,full_loader,lr=1e-2,epochs=20,verbose=True)

Epoch   19 Loss: 7.31733e-02 Accuracy: 0.98737


In [ ]:
#Training accuracy.
models.accuracy(net,meta_train_ds.samples,meta_train_ds.labels,verbose=True)

7421.0 7500


0.9894666666666667

In [ ]:
# Test accuracy.
models.accuracy(net,meta_test_ds.samples,meta_test_ds.labels)

2433.0 2500


0.9732

# Meta-Learning: Tasks

Generate a k-shot n-way loader using the meta-training dataset

In [ ]:
meta_train_kloader=KShotLoader(meta_train_ds,shots=5,ways=2)

Sample a task - each task has a k-shot n-way training set and a similar test set

In [ ]:
d_train,d_test=meta_train_kloader.get_task()

Let's try directly learning using the task training set albeit its small size: create a dataset and loader and train it with the earlier network and Train function.

In [ ]:
taskds = utils.MyDS(d_train[0],d_train[1])

In [ ]:
d_train_loader = torch.utils.data.DataLoader(dataset=taskds,batch_size=1,shuffle=True)

In [ ]:
net,losses,accs=models.Train(net,d_train_loader,lr=1e-1,epochs=10,verbose=True)

Epoch   9 Loss: 1.36185e-03 Accuracy: 1.00000


How does it do on the test set of the sampled task?

In [ ]:
models.accuracy(net,d_test[0],d_test[1])

9.0 10


0.9

# CNP-based  Meta-learning

In [ ]:
# optimisers from torch
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
lossfn = torch.nn.NLLLoss()

Get a task dataset.

In [ ]:
meta_train_kloader=KShotLoader(meta_train_ds,shots=5,ways=2,num_tasks=1000)

In [ ]:
d_train,d_test = meta_train_kloader.get_task() #training data and test data of meta_train 

In [ ]:
net = CNP(n_features=20,dims=[32,64,32],n_classes=2)

In [ ]:
print(net.mlp1,net.mlp2)

MLP(
  (layers): ModuleList(
    (0): Linear(in_features=20, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=32, bias=True)
  )
) MLP(
  (layers): ModuleList(
    (0): Linear(in_features=84, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): ReLU()
    (6): Linear(in_features=32, out_features=2, bias=True)
    (7): LogSoftmax(dim=1)
  )
)


In [ ]:
# torch.eye(2)[d_train[1]] 
# d_train[1]

tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])

In [ ]:
!cp drive/MyDrive/'Colab Notebooks'/'Meta_Learning_hw2'/CNP.ipynb .
# print(d_train[0].shape)
# print(d_train[1].shape)
r = net.adapt(d_train[0],d_train[1])
r.shape

torch.Size([10, 20])
torch.Size([10])


RuntimeError: ignored

In [ ]:
m

In [ ]:
net(d_test[0],r)

# Putting it all together: CNP-based Meta-learning
Now let's put it together in a loop - CNP model-based meta-learning algorithm:

In [1]:
final_time=0

In [3]:
# Redifning accuracy function so that it takes h - dataset context - as input since net requires it.
def accuracy(Net,X_test,y_test,h,verbose=True):
    #Net.eval()
    m = X_test.shape[0]
    y_pred = Net(X_test,h)
    _, predicted = torch.max(y_pred, 1)
    correct = (predicted == y_test).float().sum().item()
    if verbose: print(correct,m)
    accuracy = correct/m
    #Net.train()
    return accuracy

In [6]:
classes_train = [i for i in range(5)]
classes_test = [i for i in range(5)]
classes_train, classes_test

([0, 1, 2, 3, 4], [0, 1, 2, 3, 4])

In [8]:
import learn2learn as l2l
import torch.optim as optim

#Generate data - euclidean
meta_train_ds, meta_test_ds, full_loader = utils.euclideanDataset(n_samples=10000,n_features=20,n_classes=10,batch_size=32)

shots,ways = 5,5
net = CNP(n_features=20,n_classes=ways,dims=[32,64,32],lr=1e-4)
lossfn = torch.nn.NLLLoss()
meta_train_kloader=KShotLoader(meta_train_ds,shots=shots,ways=ways,num_tasks=1000, classes=classes_train)

#Meta-testing task loader for later.
meta_test_kloader=KShotLoader(meta_test_ds,shots=shots,ways=ways,classes=classes_test)

In [13]:
import numpy as np
epoch=0
n_epochs=100
task_count=50
start_time=time()
while epoch<n_epochs:
    test_loss = 0.0
    test_acc = 0.0
    # Sample and train on a task
    for task in range(task_count):
        d_train,d_test=meta_train_kloader.get_task()
        rp = torch.randperm(d_train[1].shape[0])
        d_train0=d_train[0][rp]
        d_train1=d_train[1][rp]
        x_tr = d_train0
        d_tr = x_tr 
        h= net.adapt(d_tr,d_train1)
        rp1 = torch.randperm(d_test[1].shape[0])
        d_test0=d_test[0][rp1]
        d_test1=d_test[1][rp1]
        x_ts = d_test0
        y_ts_sh = torch.zeros(x_ts.shape[0],ways)
        d_ts = x_ts 
        test_preds = net(d_ts,h)
        #train_preds = net(d_tr,h)
        # Accumulate losses over tasks - note train and test loss both included
        test_loss += lossfn(test_preds,d_test1)#+lossfn(train_preds,d_train1)
        net.eval()
        test_acc += accuracy(net,d_ts,d_test1,h,verbose=False)
        net.train()
    #Update the network weights
    print('Epoch  % 2d Loss: %2.5e Avg Acc: %2.5f'%(epoch,test_loss/task_count,test_acc/task_count))
    display.clear_output(wait=True)
    net.optimizer.zero_grad()
    test_loss.backward()
    net.optimizer.step()
    epoch+=1
time_taken=time()-start_time 




Epoch   99 Loss: 1.40191e+00 Avg Acc: 0.51040


In [14]:
final_time +=time_taken
print(np.round(final_time,3))
print(f"Time taken for {n_epochs} is: ", np.round(time_taken,3))

71.127
Time taken for 100 is:  15.965


In [9]:
# train_test_acc=[]
# loss=[]
# train_test_acc.append(test_acc/task_count)
# loss.append(test_loss/task_count)
# print(train_test_acc)
# print(loss)

In [15]:
test_acc = 0.0
task_count = 50
adapt_steps = 1
# Sample and train on a task
for task in range(task_count):
    d_train,d_test=meta_test_kloader.get_task()
    x_tr = d_train[0]
    y_tr_sh = torch.cat((torch.zeros(1,ways),torch.eye(ways)[d_train[1][1:]]))
    d_tr = x_tr #torch.cat((x_tr,y_tr_sh),1)
    h=net.adapt(d_tr,d_train[1])
    x_ts = d_test[0]
    y_ts_sh = torch.zeros(x_ts.shape[0],ways)
    d_ts = x_ts #torch.cat((x_ts,y_ts_sh),1)
    test_preds = net(d_ts,h)
    test_acc += accuracy(net,d_ts,d_test[1],h,verbose=False)
    # Done with a task
net.train()
print('Avg Acc: %2.5f'%(test_acc/task_count))

Avg Acc: 0.48000


In [11]:

# test_acc_arr=[]
# test_acc_arr=[0.2136,0.2664,0.2832,0.376]
# test_acc_arr.append(test_acc/task_count)
# epochs=[100,200,300,400]
# print(test_acc_arr)